In [1]:
!nvidia-smi

Mon Jan 13 18:10:47 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.5/112.6 GB disk)


In [4]:
from ultralytics import YOLO

from IPython.display import display, Image

In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="WnM4fwvr2D2m3k8wKSPO")
project = rf.workspace("ramee").project("runway-seg")
version = project.version(3)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to runway-seg-3 in yolov8:: 100%|██████████| 1008/1008 [00:00<00:00, 3606.64it/s]


In [6]:
model = YOLO("yolov8l-seg.pt")

100%|██████████| 88.1M/88.1M [00:00<00:00, 273MB/s]


In [7]:
results = model.train(
    data="/content/runway-seg-3/data.yaml",  # Path to the dataset YAML file
    epochs=50,                               # Reduced epochs to avoid overfitting with a small dataset
    batch=8,                                 # Reduced batch size to balance GPU memory usage and better gradients
    imgsz=512,                               # Smaller image size to reduce training time and memory requirements
    lr0=0.01,                               # Lower initial learning rate for small datasets (safer convergence)
    patience=8,                              # Lower patience for early stopping to prevent unnecessary overfitting
    weight_decay=0.001,                     # Reduced weight decay to prevent underfitting with a small dataset
    momentum=0.9,                            # Standard momentum value to maintain stability
    conf=0.2,
    augment=True ,                           # Lower confidence threshold to retain more predictions for evaluation
    name="yolov8l_seg_finetuned2",            # Updated name for better tracking
    device=0                                 # Set to '0' for GPU or 'cpu' for CPU
)

New https://pypi.org/project/ultralytics/8.3.60 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8l-seg.pt, data=/content/runway-seg-3/data.yaml, epochs=50, time=None, patience=8, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov8l_seg_finetuned2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

100%|██████████| 755k/755k [00:00<00:00, 18.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 90.7MB/s]


AMP: checks passed ✅


...

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [00:00<00:00,  2.40it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.03it/s]


                   all         47         48      0.913      0.917      0.926      0.834      0.892      0.896      0.908      0.732
Speed: 0.3ms preprocess, 14.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/segment/yolov8l_seg_finetuned2


In [8]:
 model=YOLO('/content/runs/segment/yolov8l_seg_finetuned2/weights/best.pt')


In [18]:
results = model.predict(source='/content/runway_539_jpg.rf.671f648b59a9b68d86180e43ffe26f89.jpg',save=True)


image 1/1 /content/runway_539_jpg.rf.671f648b59a9b68d86180e43ffe26f89.jpg: 512x512 1 runway-Cz6k, 126.2ms
Speed: 3.4ms preprocess, 126.2ms inference, 13.9ms postprocess per image at shape (1, 3, 512, 512)
Results saved to runs/segment/predict


In [11]:

results = model.val(data="/content/runway-seg-3/data.yaml", save=True)  # Path to your dataset


Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/runway-seg-3/valid/labels.cache... 47 images, 3 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.37s/it]


                   all         47         48      0.913      0.917      0.927      0.815      0.892      0.896      0.894      0.699
Speed: 0.3ms preprocess, 41.9ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/segment/val2


In [12]:
print("Validation Loss Breakdown:")
print(f"Box mAP@50-95: {results.box.map:.4f}")  # Box mean Average Precision (mAP) across IoU thresholds
print(f"Mask mAP@50-95: {results.seg.map:.4f}")  # Mask mean Average Precision (mAP) across IoU thresholds
print(f"Precision (Box): {results.box.mp:.4f}")  # Precision for box predictions
print(f"Recall (Box): {results.box.mr:.4f}")    # Recall for box predictions
print(f"Precision (Mask): {results.seg.mp:.4f}")  # Precision for mask predictions
print(f"Recall (Mask): {results.seg.mr:.4f}")    # Recall for mask predictions

# Speed metrics
print(f"Speed (preprocess): {results.speed['preprocess']:.1f} ms/image")
print(f"Speed (inference): {results.speed['inference']:.1f} ms/image")
print(f"Speed (postprocess): {results.speed['postprocess']:.1f} ms/image")

Validation Loss Breakdown:
Box mAP@50-95: 0.8152
Mask mAP@50-95: 0.6987
Precision (Box): 0.9131
Recall (Box): 0.9167
Precision (Mask): 0.8922
Recall (Mask): 0.8958
Speed (preprocess): 0.3 ms/image
Speed (inference): 41.9 ms/image
Speed (postprocess): 4.1 ms/image


In [13]:
import pandas as pd

# Path to the results.csv
csv_path = "/content/runs/segment/yolov8l_seg_finetuned2/results.csv"

# Try to read the CSV file
try:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Print the first row
    print("First row:")
    print(df.iloc[0])

    # Print the last row
    print("\nLast row:")
    print(df.iloc[-1])

except FileNotFoundError:
    print("The file was not found at the specified path.")


First row:
               epoch                          1
      train/box_loss                     1.2386
      train/seg_loss                     2.7168
      train/cls_loss                     2.1454
      train/dfl_loss                     1.5521
metrics/precision(B)                    0.00146
   metrics/recall(B)                    0.29167
    metrics/mAP50(B)                     0.0012
 metrics/mAP50-95(B)                    0.00039
metrics/precision(M)                    0.00021
   metrics/recall(M)                    0.04167
    metrics/mAP50(M)                     0.0001
 metrics/mAP50-95(M)                    0.00003
        val/box_loss                     3.3525
        val/seg_loss                        nan
        val/cls_loss                        inf
        val/dfl_loss                     22.323
              lr/pg0                   0.000654
              lr/pg1                   0.000654
              lr/pg2                   0.000654
Name: 0, dtype: object

Last 